<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/pr%2F1/Tarea5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC3001 Otoño 2020 Tarea 5
# Simulación con Colas y Colas de Prioridad

Nombre: Diego Román

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 22 de junio de 2020 a las 23:59

El objetivo de esta tarea es aprender a escribir programas de simulación sencillos, haciendo uso de colas y de colas de prioridad.

En primer lugar, consideremos una caja a la que van llegando clientes en instantes aleatorios, separados por intervalos de tamaño promedio $t_a$. Cada cliente que llega se pone al final de la cola, espera su turno, y cuando llega a la cabeza de la cola lo empiezan a atender. La atención dura también un tiempo aleatorio de tamaño promedio $t_s$. Cuando completa su atención, sale del sistema, y en la caja se empieza a atender al siguiente cliente de la cola, en caso de que haya alguien.

![colaT5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/colaT5.png?raw=1)

Los tiempos entre llegadas y el tiempo de atención se modelan con distribuciones exponenciales, lo cual es la suposición usual en este tipo de simulaciones.

A continuación veremos un programa que simula este proceso para un número dado de clientes y calcula el largo promedio de la cola y la demora total promedio de los clientes.

El programa lleva un reloj simulado, que comienza en cero y va  avanzando monótonamente, al siguiente instante en que un cliente llega al sistema o al siguiente instante en que un cliente sale del sistema, lo que sea que ocurra primero.

Como hay solo dos tipos posible de eventos de interés, tendremos una variable para cada uno de ellos (``evento_llegada`` y ``evento_salida``), que almacenan los respectivos tiempos. Si en este momento no hay nadie que deba salir del sistema, ``evento_salida`` es ``None``, lo cual ocurre cada vez que la cola de espera queda vacía. En cambio, ``evento_llegada`` solo es ``None``cuando ya han llegado todos los clientes que debían llegar.

Nótese que no es necesario tener generados los tiempos de todas las llegadas futuras, basta con tener el tiempo de la próxima llegada, y cuando ella se cumple se genera la siguiente.

En cada elemento que está en la cola de espera anotamos solo la hora a la que llegó ese cliente, que es lo único que necesitamos saber para poder calcular, en el momento en que sale, cuánto tiempo demoró en el sistema.

Estudie con atención este programa y la bitácora que él genera.

In [0]:
import numpy as np
class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

def print_list(L):
    for x in L:
        print(x)

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)
    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    c=Cola()
    evento_llegada=np.random.exponential(ta)
    evento_salida=None
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    
    while evento_llegada is not None or evento_salida is not None:
        # Ver si el próximo evento que toca que ocurra es la llegada o la salida de un cliente
        if (evento_salida is None) or (evento_llegada is not None and (evento_llegada<=evento_salida)):
            # Llega un cliente, avanzamos el tiempo simulado a la hora en que llega
            ahora=evento_llegada
            nclientes+=1
            
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => LLEGADA"+
                                "\n\tcola="+str(c.q))
            # El cliente que acaba de llegar se pone a la cola
            c.enq(ahora) # Basta anotar su hora de llegada
            acum_largo_cola+=c.size()
            
            if c.size()==1: # la cola estaba vacía, lo empezamos a atender de inmediato
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
            
            if nclientes<maxclientes: # programamos la hora de llegada del siguiente cliente
                evento_llegada=ahora+np.random.exponential(ta)
            else:
                evento_llegada=None
        else:               
            # Termina de atenderse un cliente, avanzamos el tiempo simulado a la hora en que sale
            ahora=evento_salida
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => SALIDA"+
                                "\n\tcola="+str(c.q))
            tllegada=c.deq() # sacamos al cliente de la cola
            acum_tiempo_en_sistema+=(ahora-tllegada)
            
            if c.is_empty(): # no hay nadie en la cola
                evento_salida=None
            else: # Hay un cliente esperando, empezamos a atenderlo
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
    
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)            

In [187]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722


In [188]:
print_list(bitacora)

T=21.25986576184801 evento_llegada=21.25986576184801 evento_salida=None => LLEGADA
	cola=[]
T=78.83677538497456 evento_llegada=78.83677538497456 evento_salida=94.24603167524747 => LLEGADA
	cola=[21.25986576184801]
T=94.24603167524747 evento_llegada=232.71542283099382 evento_salida=94.24603167524747 => SALIDA
	cola=[78.83677538497456, 21.25986576184801]
T=207.7973648436669 evento_llegada=232.71542283099382 evento_salida=207.7973648436669 => SALIDA
	cola=[78.83677538497456]
T=232.71542283099382 evento_llegada=232.71542283099382 evento_salida=None => LLEGADA
	cola=[]
T=256.5855663702635 evento_llegada=265.07595569718336 evento_salida=256.5855663702635 => SALIDA
	cola=[232.71542283099382]
T=265.07595569718336 evento_llegada=265.07595569718336 evento_salida=None => LLEGADA
	cola=[]
T=386.489172340086 evento_llegada=582.4168679105603 evento_salida=386.489172340086 => SALIDA
	cola=[265.07595569718336]
T=582.4168679105603 evento_llegada=582.4168679105603 evento_salida=None => LLEGADA
	cola=[]


## Lo que usted tiene que hacer

### 1. Mejorar este programa introduciendo una cola de prioridad

El programa está escrito sabiendo que puede haber dos tipos de eventos por ocurrir (LLEGADA y SALIDA) y que en todo momento puede haber a lo más solo uno de cada tipo.

En una situación más general, podría haber eventos de muchos tipos programados para ocurrir, en muchos instantes distintos. En ese caso, la implementación presentada es demasiado limitada, y se requiere un mecanismo más general para administrar esa lista de eventos futuros. Una *cola de prioridad* (usando el tiempo como prioridad, y en que el mínimo tiempo equivale a mejor prioridad) es la estructura adecuada, porque en esa lista de eventos futuros siempre vamos extrayendo el primero que debe ocurrir (el mínimo), y durante la ejecución se pueden generar nuevos elementos programados para ocurrir en el futuro (inserciones).

En preparación a poder simular modelos más complejos, lo que usted tiene que hacer es implementar una lista de eventos futuros mediante uan cola de prioridad, en la cual se almacenen pares de la forma

$$
(\text{tiempo},\text{tipo de evento})
$$

y modificar el programa para que utilice esa cola de prioridad en lugar de las variables ``evento_salida`` y ``evento_llegada``. Implemente la cola de prioridad en base al código para heaps que aparece en el apunte.

Sugerencias: Antes del while, encolar el primer evento de llegada, luego en cada iteración sacar un evento de la cola de prioridad y procesarlo, y cuando en el programa original se generaba un evento nuevo y se asignaba a la variable respectiva, ahora hay que insertar un nuevo evento del tipo correspondiente en la cola de prioridad.

El programa resultante debiera ser más simple, porque varias cosas se unifican al verlas de esta manera.

Ejecute ambos programas en modo ``debug`` y asegúrese que en las bitácoras resultantes los eventos que ocurren y sus tiempos sean idénticos.

In [0]:
import numpy as np
#modificadas para que puedan comparar los tiempos de los pares (tiempo, tipo de evento)
#y para que mayor prioridad = menor tiempo 
def trepar(a,j): # El elemento a[j] trepa hasta su nivel de prioridad 
    while j>=1 and a[j][0]<a[(j-1)//2][0]:
        (a[j],a[(j-1)//2])=(a[(j-1)//2],a[j]) # intercambiamos con el padre
        j=(j-1)//2 # subimos al lugar del padre
        
def hundir(a,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
    while 2*j+1<n: # mientras tenga al menos 1 hijo
        k=2*j+1 # el hijo izquierdo
        if k+1<n and a[k+1][0]<a[k][0]: # el hijo derecho existe y es MENOR
            k+=1
        if a[j][0]<=a[k][0]: # tiene mejor prioridad que ambos hijos
            break
        (a[j],a[k])=(a[k],a[j]) # se intercambia con el mayor de los hijos
        j=k # bajamos al lugar del mayor de los hijos
    
class Heap:
    def __init__(self,maxn=100):
        self.a=np.zeros(maxn, dtype='O')
        self.n=0
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x    
        trepar(self.a,self.n)
        self.n+=1       
    def extract_max(self): #maxima prioridad, minimo tiempo
        x=self.a[0] # esta variable lleva el máximo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        hundir(self.a,0,self.n)
        return x
    def imprimir(self):
        print(self.a[0:self.n])
    def is_empty(self):
         if self.n == 0: return True
         else: return False
    def size(self):
        return self.n

def print_list(L):
    for x in L:
        print(x)

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)
    ahora = 0 # tiempo simulado
    n_clientes = 0 # número de clientes que han llegado
    c=Heap(maxn=maxclientes)
    tiempo_llegada=np.random.exponential(ta)
    tiempo_salida = tiempo_llegada + np.random.exponential(ts)
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    
    primer_evento_llegada = (tiempo_llegada, 'LLEGADA')
    c.insert(primer_evento_llegada) #llegó 1 cliente
    primer_evento_salida = (tiempo_salida, 'SALIDA')
    c.insert(primer_evento_salida)
    #simulacion
    ahora = tiempo_llegada
    n_fila = 0

    while n_clientes < maxclientes:
        cola_string = str(c.a[:c.n])
        #Atender siempre a la máxima prioridad (es decir, extraer el mínimo tiempo en el Heap)
        evento_actual = c.extract_max()
        ahora = evento_actual[0]
        tipo_evento = evento_actual[1]
        # ¿Toca procesar una salida de la cola o la entrada de alguien?
        if tipo_evento == 'LLEGADA':
            n_clientes += 1 #llegó un nuevo cliente
            n_fila += 1
            acum_largo_cola += n_fila

        else: #tipo_evento == 'SALIDA'
            n_fila -= 1

            #se está procesando una transacción, se programa la salida del siguiente
            tiempo_nueva_salida = ahora + np.random.exponential(ts)
            c.insert((tiempo_nueva_salida, 'SALIDA'))
            
            #tiempo de atención
            tiempo_atencion = tiempo_nueva_salida - ahora
            acum_tiempo_en_sistema += tiempo_atencion

        if debug:
            bitacora.append("T="+str(ahora)+ " => " + str(tipo_evento) + "\n\tcola="+cola_string+'\n')
        
        #se genera una nueva entrada (futura)
        if tipo_evento == 'SALIDA': ahora += tiempo_atencion
        tiempo_nueva_entrada = ahora + np.random.exponential(ta)
        c.insert((tiempo_nueva_entrada, 'LLEGADA'))
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)            

In [190]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 3.1  Tiempo promedio en el sistema= 37.58274169780943


In [191]:
print_list(bitacora)

T=21.25986576184801 => LLEGADA
	cola=[(21.25986576184801, 'LLEGADA') (94.24603167524747, 'SALIDA')]

T=78.83677538497456 => LLEGADA
	cola=[(78.83677538497456, 'LLEGADA') (94.24603167524747, 'SALIDA')]

T=94.24603167524747 => SALIDA
	cola=[(94.24603167524747, 'SALIDA') (232.71542283099382, 'LLEGADA')]

T=207.7973648436669 => SALIDA
	cola=[(207.7973648436669, 'SALIDA') (232.71542283099382, 'LLEGADA')
 (239.62422289602654, 'LLEGADA')]

T=232.06776449330906 => SALIDA
	cola=[(232.06776449330906, 'SALIDA') (239.62422289602654, 'LLEGADA')
 (232.71542283099382, 'LLEGADA') (393.95205335051253, 'LLEGADA')]

T=232.71542283099382 => LLEGADA
	cola=[(232.71542283099382, 'LLEGADA') (239.62422289602654, 'LLEGADA')
 (393.95205335051253, 'LLEGADA') (470.07344865334176, 'SALIDA')
 (678.7665079218352, 'LLEGADA')]

T=239.62422289602654 => LLEGADA
	cola=[(239.62422289602654, 'LLEGADA') (277.00366178666235, 'LLEGADA')
 (393.95205335051253, 'LLEGADA') (678.7665079218352, 'LLEGADA')
 (470.07344865334176, 'SALI

## 2. Modificar el modelo para introducir un tiempo comprando

A continuación, introduciremos un elemento adicional en el modelo, y usted debe modificar su programa para incluirlo.

Supondremos que una vez que un cliente llega, en lugar de ponerse a la cola de inmediato, pasa un rato en la tienda comprando, y una vez que termina (después de transcurrido un tiempo aleatorio de duración promedio $t_c$), en ese momento recién se pone a la cola para pagar en la caja.

![cola2T5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/cola2T5.png?raw=1)

Observe que, como puede haber muchos clientes en la tienda que aún no terminan de comprar, en la lista de eventos futuros ahora puede haber muchos eventos de un nuevo tipo (FINCOMPRA), lo que justifica el haber introducido la cola de prioridad.

Ejecute su programa en modo ``debug`` agregando un parámetro $t_c=300$ e imprima el largo promedio de la cola, la demora total promedio de los clientes y la bitácora.

## 3. ¿Qué hay que entregar?

Usted debe entregar este mismo archivo, modificado de acuerdo a lo que se pide. Haga todos los cambios necesarios para explicar y documentar adecuadamente su código. No olvide poner su nombre.